In [ ]:
import os
import json
import uuid
import datetime
import requests
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import chromadb
import google.generativeai as genai


In [ ]:
from dotenv import load_dotenv
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [ ]:
# Load FAQs
with open('faq.json', 'r') as f:
    data = json.load(f)

documents = [f"Q: {item['question']}\nA: {item['answer']}" for item in data]
metadatas = [{"question": item["question"]} for item in data]

# Embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(documents, show_progress_bar=True)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:

# Setup ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_faq")
collection = chroma_client.get_or_create_collection(name="faq")

if len(collection.get()['ids']) == 0:
    collection.add(
        documents=documents,
        embeddings=embeddings.tolist(),
        metadatas=metadatas,
        ids=[str(i) for i in range(len(documents))]
    )

# Load Gemini model
model = genai.GenerativeModel("gemini-1.5-flash")
chat = model.start_chat(history=[])


In [ ]:
# Emotion Detection Model
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

def detect_emotion(text):
    emotions = emotion_classifier(text)[0]
    top_emotion = max(emotions, key=lambda x: x['score'])
    return top_emotion['label'], top_emotion['score']

def add_empathy_to_response(emotion, base_response):
    empathy_prefix = {
        "joy": "That's wonderful to hear! 😊 ",
        "anger": "I'm sorry you're feeling upset. Let's work on this together. ",
        "sadness": "I'm here for you. 💙 ", 
        "fear": "Don't worry, I'm here to help. ",
        "surprise": "That's interesting! ",
        "neutral": ""
    }
    return empathy_prefix.get(emotion, "") + base_response


Device set to use cpu


In [ ]:
def generate_gemini_answer(query, k=3, similarity_threshold=0.6):
    query_embedding = embedder.encode([query])[0]
    results = collection.query(query_embeddings=[query_embedding], n_results=k)

    if not results['documents'][0] or len(results['documents'][0]) == 0:
        return None, 0.0  # Return None with low confidence

    context = "\n\n".join(results['documents'][0])
    
    emotion, _ = detect_emotion(query)
    prompt = f"""You are a helpful and emotionally aware assistant. The user appears to be feeling {emotion} based on the query. Respond with professionalism, empathy, and clarity.

        If the emotion is 'anger', gently acknowledge their frustration and offer to escalate the issue by asking if they'd like to raise a support ticket.
        Dont use emojis.
        Context:
        {context}

        Question: {query}
        Answer:"""

    response = model.generate_content(prompt)
    avg_score = 1 - sum(results['distances'][0]) / len(results['distances'][0])
    return response.text, avg_score


In [ ]:

def notify_human_agent(session_id, user_input):
    payload = {
        "session_id": session_id,
        "message": user_input,
        "status": "escalation_triggered"
    }
    try:
        response = requests.post("https://your-agent-alert-endpoint.com/notify", json=payload)
        if response.status_code == 200:
            print("Bot: Human agent has been notified successfully.")
        else:
            print("Bot: Failed to notify human agent.")
    except Exception as e:
        print(f"Bot: Notification failed due to: {e}")


In [ ]:
def ask_csat_feedback(summary_text):
    try:
        rating_input = input("Please rate your experience with this chat (1-5): ").strip()
        text_feedback = input("Optional: Share any comments about your experience: ").strip()

        if rating_input == "":
            raise ValueError("No rating provided")

        rating = int(rating_input)
        return rating, text_feedback or "No comment provided by user."

    except ValueError:
        print("No valid CSAT rating provided. Generating CSAT from summary...")

        auto_eval_prompt = f"""
        Based on the following chat summary, give a customer satisfaction score between 1 (very bad) and 5 (excellent).
        Also provide a short comment representing what a typical user might feel.

        Summary:
        {summary_text}

        Return response in the following format:
        Rating: <number>
        Comment: <generated comment>
        """

        auto_response = model.generate_content(auto_eval_prompt).text.strip()

        # Extracting score and comment
        lines = auto_response.splitlines()
        rating_line = next((line for line in lines if line.lower().startswith("rating:")), "Rating: 3")
        comment_line = next((line for line in lines if line.lower().startswith("comment:")), "Comment: Not bad.")

        try:
            rating = int(rating_line.split(":")[1].strip())
        except:
            rating = 3

        comment = comment_line.split(":", 1)[1].strip() if ":" in comment_line else "No comment."
        return rating, comment


In [ ]:

# CHAT SESSION
session_id = str(uuid.uuid4())
session_start_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
chat_history_log = []
CONFIDENCE_THRESHOLD = 0


In [ ]:
print(f"🤖 Gemini EQ Chatbot (Session ID: {session_id}) - Type 'exit' to quit")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break

    emotion, confidence = detect_emotion(user_input)
    chat_history_log.append(f"User: {user_input} [Emotion: {emotion} ({confidence:.2f})]")

    rag_response, confidence_score = generate_gemini_answer(user_input)

    if rag_response:
        empathetic_response = add_empathy_to_response(emotion, rag_response)
        print("Bot (EQ-RAG):", rag_response)
        chat_history_log.append(f"Bot (EQ-RAG): {empathetic_response} [Confidence: {confidence_score:.2f}]")
    else:
        print("Bot: I'm not confident in my answer. Escalating to a human agent... 🧑‍💼")
        chat_history_log.append("Bot: Escalation triggered due to no response.")
        notify_human_agent(session_id, user_input)


🤖 Gemini EQ Chatbot (Session ID: 797a9cd1-b55a-43e0-8b88-10b8979985cc) - Type 'exit' to quit
Bot (EQ-RAG): To provide you with your order summary, I need more information.  Could you please provide your order number or the email address associated with your order?  This will allow me to access your order details and provide you with the information you need.

Bot (EQ-RAG): A:  We strive to provide accurate and timely information.  Is there anything else I can help you with today?

Bot (EQ-RAG): I understand your frustration and disappointment that your question wasn't answered to your satisfaction.  I apologize for that.  Could you please rephrase your question or provide more details about what you were hoping to learn?  The more information you can give me, the better I can assist you.

Bot (EQ-RAG): I understand your frustration.  It sounds like you're feeling angry, and I apologize if my previous responses haven't adequately addressed your needs.  Could you please tell me more abou

In [ ]:
# Save chat history
with open(f"chat_history_{session_id}.txt", "w", encoding="utf-8") as f:
    f.write(f"Session ID: {session_id}\nStart Time: {session_start_time}\n\n")
    for line in chat_history_log:
        f.write(line + "\n")




In [ ]:
# Summarize chat
with open(f"chat_history_{session_id}.txt", "r", encoding="utf-8") as f:
    chat_text = f.read()



In [ ]:
csat_rating, csat_comment = ask_csat_feedback(summary_response.text)
print(f"Thank you for your feedback! You rated this chat: {csat_rating}/5")
if csat_comment:
    print(f"Your comment: {csat_comment}")

with open(f"csat_{session_id}.txt", "w", encoding="utf-8") as f:
    f.write(f"Session ID: {session_id}\nCSAT Rating: {csat_rating}\nComment: {csat_comment}\n")


No valid CSAT rating provided. Generating CSAT from summary...
Thank you for your feedback! You rated this chat: 2/5
Your comment: Completely unhelpful and frustrating.  The bot couldn't even track my order, and then made a bizarre and irrelevant response. I'm glad I could escalate to a human, but the whole experience was a waste of time.
